In [1]:
# import h5py
import config
import matplotlib.pyplot as plt
from Models import FeedForward, LSTMPredictor, BatchOneLayerLSTM
from ModelTrainer import ModelTrainer
import numpy as np
import os
import pandas as pd
import pickle
import random
import time
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, roc_curve

/home/subhankar/.conda/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [2]:
# hdf_file_path: Loan_Data
key_len = 1635
random.seed = 448
indices = [i for i in range(key_len)]

train_lim = int(0.7*key_len)
test_lim = train_lim + int(0.25*key_len)
valid_lim = test_lim + int((1 - train_lim - test_lim)*key_len)
train, test, valid = indices[:train_lim], indices[train_lim: test_lim], indices[test_lim:]

In [3]:
len(train), len(test), len(valid)

(1144, 408, 83)

In [ ]:
# Model Parameters
INPUT_SIZE = 110
HIDDEN_LAYER1 = 70
HIDDEN_LAYER2 = 55
NUM_CLASSES = len(config.unique_values['current_status']) # 45
# Training paramters
LEARNING_RATE = 0.001
NUM_EPOCHS = 2
CONCERNED_CLASS = 1
DATA_FILE = 'Loan_Data.h5'

In [ ]:
model = FeedForward(INPUT_SIZE, HIDDEN_LAYER1, HIDDEN_LAYER2, NUM_CLASSES)

In [ ]:
trainer = ModelTrainer(model, LEARNING_RATE, NUM_EPOCHS, CONCERNED_CLASS, DATA_FILE)

In [ ]:
trainer.trainFF(train, test)

In [ ]:
torch.cuda.is_available()

In [ ]:
trainer.training_auc

In [ ]:
trainer.test_auc

In [ ]:
trainer.loss_values

In [ ]:
trainer.training_accuracy

In [ ]:
trainer.loss_values

### LSTM

In [4]:
# hdf_file_path: Loan_Data
key_len = 1635
random.seed = 448
rand_ind = random.sample([i for i in range(key_len)], key_len)

train_lim = int(0.7*key_len)
test_lim = train_lim + int(0.25*key_len)
valid_lim = test_lim + int((1 - train_lim - test_lim)*key_len)
train, test, valid = rand_ind[:train_lim], rand_ind[train_lim: test_lim], rand_ind[test_lim:]

In [10]:
# Model Parameters
INPUT_DIM = 110
HIDDEN_DIM = 128
NUM_EPOCHS = 2
CONCERNED_CLASS = 43
NUM_CLASS = len(config.unique_values['current_status'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Training paramters
LEARNING_RATE = 0.001
NUM_EPOCHS = 1
CONCERNED_CLASS = 1
DATA_FILE = 'Loan_Data.h5'

In [11]:
model = BatchOneLayerLSTM(INPUT_DIM, HIDDEN_DIM, NUM_CLASS, device)

In [12]:
trainer = ModelTrainer(model, LEARNING_RATE, NUM_EPOCHS, CONCERNED_CLASS, DATA_FILE, device)

In [13]:
for i in list(model.parameters()):
    print(torch.norm(i))

tensor(12.1032, grad_fn=<NormBackward0>)
tensor(13.0444, grad_fn=<NormBackward0>)
tensor(1.1312, grad_fn=<NormBackward0>)
tensor(1.1611, grad_fn=<NormBackward0>)
tensor(3.8951, grad_fn=<NormBackward0>)
tensor(0.3693, grad_fn=<NormBackward0>)


In [ ]:
trainer.trainLSTM(train, test)

/home/subhankar/d/GitHub/IndependentStudy/Code/DataGen.py:31: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  df.current_status = df.current_status.astype('category', categories=config.unique_values['current_status'])
/home/subhankar/d/GitHub/IndependentStudy/Code/DataGen.py:34: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  df.state = df.state.astype('category', categories=config.unique_values['unique_states'])
/home/subhankar/d/GitHub/IndependentStudy/Code/ModelTrainer.py:141: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  auc = self.GetAUCScore(y_true, F.log_softmax(outputs).data.cpu().numpy(), self.concerned_class)


Epoch: [1/1], Step: [1/1144], Loss: 2.1432, Roc: 0.4811
Epoch: [1/1], Step: [101/1144], Loss: 0.0336, Roc: 0.4977
Epoch: [1/1], Step: [201/1144], Loss: 0.0226, Roc: 0.4958
Epoch: [1/1], Step: [301/1144], Loss: 0.0196, Roc: 0.4952


In [ ]:
trainer.training_auc

In [ ]:
trainer.test_auc

In [ ]:
trainer.loss_values